In [1]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd

# ---- Setup ----
scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive"
]

creds = ServiceAccountCredentials.from_json_keyfile_name(
    "eighth-duality-453409-p4-4d48bf6ca4d8.json", scope)
client = gspread.authorize(creds)

# 🔁 Loop through all product sheets
product_sheets = [f'Product_{i}' for i in range(1, 6)]
sheet_id = "1RVhvyjjI36WHccksZ3skgMV7PL2Nt0CgEwPoHMzQfTI"

dfs = {}

for sheet_name in product_sheets:
    worksheet = client.open_by_key(sheet_id).worksheet(sheet_name)
    data = worksheet.get_all_records()
    df = pd.DataFrame(data)
    df['Date'] = pd.to_datetime(df['Date'])  # Ensure proper datetime format
    df = df.sort_values('Date')
    dfs[sheet_name] = df

print("✅ All product sheets loaded from Google Sheets.")


✅ All product sheets loaded from Google Sheets.


In [ ]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping


# Sliding window generator
def create_sequences(data, window_size, output_size):
    X, y = [], []
    for i in range(len(data) - window_size - output_size + 1):
        X.append(data[i:i+window_size])
        y.append(data[i+window_size:i+window_size+output_size])
    return np.array(X), np.array(y)

# Settings
WINDOW_SIZE = 60
OUTPUT_SIZE = 30
BATCH_SIZE = 32
EPOCHS = 100

for sheet, df in dfs.items():
    print(f"\n🚀 Processing {sheet}...")

    # Use only 'Units Sold' column and scale
    values = df['Units_Sold'].values.reshape(-1, 1)
    scaler = MinMaxScaler()
    values_scaled = scaler.fit_transform(values)

    # Create sequences
    X, y = create_sequences(values_scaled, WINDOW_SIZE, OUTPUT_SIZE)
    print(f"X shape: {X.shape}, y shape: {y.shape}")

    # Train/validation/test split
    train_size = int(0.8 * len(X))
    val_size = int(0.1 * len(X))
    X_train, y_train = X[:train_size], y[:train_size]
    X_val, y_val = X[train_size:train_size+val_size], y[train_size:train_size+val_size]
    X_test, y_test = X[train_size+val_size:], y[train_size+val_size:]

    # Reshape for LSTM
    X_train = X_train.reshape((-1, WINDOW_SIZE, 1))
    X_val = X_val.reshape((-1, WINDOW_SIZE, 1))
    X_test = X_test.reshape((-1, WINDOW_SIZE, 1))

    # LSTM model
    model = Sequential([
        LSTM(128, return_sequences=True, input_shape=(WINDOW_SIZE, 1)),
        Dropout(0.2),
        LSTM(64),
        Dense(OUTPUT_SIZE)
    ])

    model.compile(optimizer='adam', loss='mse', metrics=['mse'])

    # Early stopping
    early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Train
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        callbacks=[early_stop],
        verbose=1
    )
    print(f"y_test shape: {y_test.shape}, y_pred shape: {y_pred.shape}")


    # Predict and inverse scale
    y_pred = model.predict(X_test)
    y_pred_inv = scaler.inverse_transform(y_pred.reshape(-1, 1)).reshape(y_pred.shape)
    y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1)).reshape(y_test.shape)

    # Evaluation
    mse = mean_squared_error(y_test_inv.flatten(), y_pred_inv.flatten())
    mae = mean_absolute_error(y_test_inv.flatten(), y_pred_inv.flatten())
    rmse = np.sqrt(mse)

    print(f"{sheet} - MAE: {mae:.2f}, MSE: {mse:.2f}, RMSE: {rmse:.2f}")



🚀 Processing Product_1...
X shape: (4911, 60, 1), y shape: (4911, 30, 1)
Epoch 1/100


d:\AI\Mastery series\Summer 2025\Smartinventory\inventory\lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


123/123 ━━━━━━━━━━━━━━━━━━━━ 5s 26ms/step - loss: 0.0517 - mse: 0.0517 - val_loss: 0.0183 - val_mse: 0.0183
Epoch 2/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - loss: 0.0165 - mse: 0.0165 - val_loss: 0.0176 - val_mse: 0.0176
Epoch 3/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - loss: 0.0164 - mse: 0.0164 - val_loss: 0.0176 - val_mse: 0.0176
Epoch 4/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - loss: 0.0164 - mse: 0.0164 - val_loss: 0.0181 - val_mse: 0.0181
Epoch 5/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - loss: 0.0164 - mse: 0.0164 - val_loss: 0.0181 - val_mse: 0.0181
Epoch 6/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 4s 33ms/step - loss: 0.0163 - mse: 0.0163 - val_loss: 0.0176 - val_mse: 0.0176
Epoch 7/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 4s 36ms/step - loss: 0.0163 - mse: 0.0163 - val_loss: 0.0177 - val_mse: 0.0177
Epoch 8/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step - loss: 0.0164 - mse: 0.0164 - val_loss: 0.0178 - val_mse: 0.0178
Epoch 9/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 4s 36ms/ste

d:\AI\Mastery series\Summer 2025\Smartinventory\inventory\lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


123/123 ━━━━━━━━━━━━━━━━━━━━ 7s 44ms/step - loss: 0.0498 - mse: 0.0498 - val_loss: 0.0173 - val_mse: 0.0173
Epoch 2/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 5s 40ms/step - loss: 0.0166 - mse: 0.0166 - val_loss: 0.0174 - val_mse: 0.0174
Epoch 3/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 5s 38ms/step - loss: 0.0164 - mse: 0.0164 - val_loss: 0.0174 - val_mse: 0.0174
Epoch 4/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step - loss: 0.0165 - mse: 0.0165 - val_loss: 0.0180 - val_mse: 0.0180
Epoch 5/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step - loss: 0.0163 - mse: 0.0163 - val_loss: 0.0175 - val_mse: 0.0175
Epoch 6/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step - loss: 0.0162 - mse: 0.0162 - val_loss: 0.0174 - val_mse: 0.0174
Epoch 7/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 4s 36ms/step - loss: 0.0164 - mse: 0.0164 - val_loss: 0.0176 - val_mse: 0.0176
Epoch 8/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 5s 37ms/step - loss: 0.0162 - mse: 0.0162 - val_loss: 0.0174 - val_mse: 0.0174
Epoch 9/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 5s 39ms/ste

...

123/123 ━━━━━━━━━━━━━━━━━━━━ 6s 37ms/step - loss: 0.0484 - mse: 0.0484 - val_loss: 0.0195 - val_mse: 0.0195
Epoch 2/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 4s 34ms/step - loss: 0.0184 - mse: 0.0184 - val_loss: 0.0195 - val_mse: 0.0195
Epoch 3/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step - loss: 0.0182 - mse: 0.0182 - val_loss: 0.0193 - val_mse: 0.0193
Epoch 4/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step - loss: 0.0183 - mse: 0.0183 - val_loss: 0.0195 - val_mse: 0.0195
Epoch 5/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step - loss: 0.0183 - mse: 0.0183 - val_loss: 0.0197 - val_mse: 0.0197
Epoch 6/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step - loss: 0.0179 - mse: 0.0179 - val_loss: 0.0193 - val_mse: 0.0193
Epoch 7/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step - loss: 0.0182 - mse: 0.0182 - val_loss: 0.0198 - val_mse: 0.0198
Epoch 8/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step - loss: 0.0181 - mse: 0.0181 - val_loss: 0.0199 - val_mse: 0.0199
Epoch 9/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/ste

d:\AI\Mastery series\Summer 2025\Smartinventory\inventory\lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


123/123 ━━━━━━━━━━━━━━━━━━━━ 6s 36ms/step - loss: 0.0549 - mse: 0.0549 - val_loss: 0.0208 - val_mse: 0.0208
Epoch 2/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 4s 34ms/step - loss: 0.0185 - mse: 0.0185 - val_loss: 0.0207 - val_mse: 0.0207
Epoch 3/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 4s 34ms/step - loss: 0.0183 - mse: 0.0183 - val_loss: 0.0211 - val_mse: 0.0211
Epoch 4/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 4s 34ms/step - loss: 0.0183 - mse: 0.0183 - val_loss: 0.0205 - val_mse: 0.0205
Epoch 5/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 4s 34ms/step - loss: 0.0181 - mse: 0.0181 - val_loss: 0.0198 - val_mse: 0.0198
Epoch 6/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 4s 34ms/step - loss: 0.0184 - mse: 0.0184 - val_loss: 0.0201 - val_mse: 0.0201
Epoch 7/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 4s 36ms/step - loss: 0.0181 - mse: 0.0181 - val_loss: 0.0199 - val_mse: 0.0199
Epoch 8/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step - loss: 0.0182 - mse: 0.0182 - val_loss: 0.0204 - val_mse: 0.0204
Epoch 9/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/ste

d:\AI\Mastery series\Summer 2025\Smartinventory\inventory\lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


123/123 ━━━━━━━━━━━━━━━━━━━━ 7s 37ms/step - loss: 0.0517 - mse: 0.0517 - val_loss: 0.0213 - val_mse: 0.0213
Epoch 2/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step - loss: 0.0189 - mse: 0.0189 - val_loss: 0.0209 - val_mse: 0.0209
Epoch 3/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step - loss: 0.0189 - mse: 0.0189 - val_loss: 0.0206 - val_mse: 0.0206
Epoch 4/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step - loss: 0.0187 - mse: 0.0187 - val_loss: 0.0207 - val_mse: 0.0207
Epoch 5/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step - loss: 0.0185 - mse: 0.0185 - val_loss: 0.0208 - val_mse: 0.0208
Epoch 6/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step - loss: 0.0188 - mse: 0.0188 - val_loss: 0.0215 - val_mse: 0.0215
Epoch 7/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step - loss: 0.0186 - mse: 0.0186 - val_loss: 0.0203 - val_mse: 0.0203
Epoch 8/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step - loss: 0.0188 - mse: 0.0188 - val_loss: 0.0203 - val_mse: 0.0203
Epoch 9/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/ste